In [7]:
# Import des données
import pandas as pd 
import numpy as np

# import du modèle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler

In [8]:
# import des données
df = pd.read_csv("../data/silver_data.csv")
display(df.head())
print(df.shape)
print(df.columns)

,jobs,description,loc,datePosted,workhours,salary,contract_type,experience,diploma,skills,qualifications,industry,company,size,company_description,tools,ID_dep,ville,date
0,Data Engineer sénior (F/H) CDI (H/F),"En tant que Data Engineer chez Quantmetry, vou...",75 - PARIS 08,Actualisé le 12 juillet 2023,38H Travail en journée,40000.0 - 50000.0,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
1,Data Consultant Stratégie Sénior (H/F),Nous recrutons des personnes avec une appétenc...,75 - PARIS 08,Actualisé le 12 juillet 2023,35H Travail en journée,3100.0 - 5000.0,Contrat à durée indéterminée,5 ans,NaN,"analyser les résultats d'un projet, décliner l...",Cadre,Conseil pour les affaires et autres conseils d...,QUANTMETRY,100 à 199 salariés,Pure player en Data et Intelligence Artificiel...,NaN,75,PARIS 08,2023/07/12
2,Chef de projets Performance Durable/Energie/Da...,Intégré(e) au sein de la Direction Performance...,92 - ASNIERES SUR SEINE,Actualisé le 13 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,3 ans,NaN,contrôler et faire appliquer le respect de dis...,Employé qualifié,Activités des sièges sociaux,NEXITY,250 à 499 salariés,Nexity est aujourd hui leader sur les différen...,tableau,92,ASNIERES SUR SEINE,2023/07/13
3,Ingénieur Data H/F,"Vos missions\nAu sein de projets Agile Scrum, ...",66 - PERPIGNAN,Actualisé le 19 juillet 2023,35H Travail en journée,NaN,Contrat à durée indéterminée,5 ans,NaN,"concevoir et gérer un projet, concevoir un log...",Cadre,"Autres intermédiaires du commerce en denrées, ...",SOCIETE DE DISTRIBUTION DE PRODUITS MARA,100 à 199 salariés,"Azura, est un groupe familial, en forte croiss...","tableau, sas",66,PERPIGNAN,2023/07/19
4,Data Engineer pour la Qualité Totale H/F (H/F),Vous êtes Rattaché(e) au Responsable de l'équi...,92 - ST CLOUD,Actualisé le 19 juillet 2023,35H Travail en journée,38000.0 - 70000.0,Contrat à durée indéterminée,5 ans,NaN,"application web, déterminer des mesures correc...",Cadre,Activités des sièges sociaux,DASSAULT AVIATION,Non employeur (pas de salarié au cours de l'an...,Dassault Aviation,NaN,92,ST CLOUD,2023/07/19


(3299, 19)
Index(['jobs', 'description', 'loc', 'datePosted', 'workhours', 'salary',
       'contract_type', 'experience', 'diploma', 'skills', 'qualifications',
       'industry', 'company', 'size', 'company_description', 'tools', 'ID_dep',
       'ville', 'date'],
      dtype='object')


In [9]:
def get_recommendations(input_title):
    input_movie = df[df['jobs'] == input_title]
    
    features = input_movie[['loc', 'experience', 'tools', 'skills']]
    
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(df['loc'] + " " + df['experience'] + " " + df['tools'] + " " + df['skills'])
    
    scaler = MinMaxScaler()
    numerical_features = scaler.fit_transform(df[['imdb_score', 'duration']])
    
    combined_features = pd.concat([pd.DataFrame(tfidf_matrix.toarray()), pd.DataFrame(numerical_features)], axis=1)
    
    cosine_sim = cosine_similarity(combined_features, combined_features)
    
    idx = df.index[df['jobs'] == input_title].tolist()[0]
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    sim_indices = [i[0] for i in sim_scores[1:6]]  
    
    return df['jobs'].iloc[sim_indices]